# Carga de Dados para a Camada Gold
Este notebook orquestra o processo de ETL para carregar os dados de energia da camada `curated` para a camada `gold` do nosso Data Warehouse.

### 1. Imports

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

RAW_DIR = Path("data/raw")
CURATED_DIR = Path("data/curated")
DB_PATH = Path("db/database.db")

### 2. Criação das Tabelas no Banco de Dados
Lê e executa o script SQL para criar as tabelas de `staging` e `gold`, estabelecendo o schema no banco de dados.

In [2]:
with open("sql/local/01_create_tables.sql") as f:
    create_tables_sql = f.read()

conn = sqlite3.connect(DB_PATH)
conn.executescript(create_tables_sql)
conn.commit()

print("Tabelas criadas com sucesso!")

Tabelas criadas com sucesso!


In [3]:
# Checar se as tabelas foram criadas e o schema está correto
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = conn.execute(tables_query).fetchall()
for table in tables:
    print(f"Tabela criada: {table[0]}")

Tabela criada: balanco_subsistema_stg
Tabela criada: fact_balanco_subsistema


### 3. Verificação do Schema
Verifica se as tabelas foram criadas corretamente e exibe o schema da tabela de staging para confirmação.

In [4]:
schema_query = "PRAGMA table_info(balanco_subsistema_stg);"
schema = conn.execute(schema_query).fetchall()
print("Schema da tabela balanco_subsistema_stg:")
for column in schema:
    print(column)

Schema da tabela balanco_subsistema_stg:
(0, 'din_instante', 'TEXT', 1, None, 0)
(1, 'id_subsistema', 'TEXT', 1, None, 0)
(2, 'nom_subsistema', 'TEXT', 1, None, 0)
(3, 'geracao_hidraulica', 'REAL', 0, None, 0)
(4, 'geracao_termica', 'REAL', 0, None, 0)
(5, 'geracao_eolica', 'REAL', 0, None, 0)
(6, 'geracao_solar', 'REAL', 0, None, 0)
(7, 'carga', 'REAL', 1, None, 0)
(8, 'intercambio', 'REAL', 1, None, 0)
(9, 'dt_ingestao', 'TEXT', 1, None, 0)


### 4. Leitura dos Dados Curados
Lê os dados do arquivo Parquet da camada `curated`, que já foram processados e estão prontos para serem carregados.

In [5]:
parquet_path = "data/curated/balanco_subsistema/year=2024/balanco_subsistema_2024.parquet"

df = pd.read_parquet(parquet_path)
df.head()

,din_instante,id_subsistema,nom_subsistema,geracao_hidraulica,geracao_termica,geracao_eolica,geracao_solar,carga,intercambio,dt_ingestao
0,2024-01-01,NE,NORDESTE,3503.617920,486.473999,2491.562988,1.550,11976.884,-5493.679,2025-11-24T21:15:53Z
1,2024-01-01,N,NORTE,3006.385010,2469.684082,2.193000,0.000,6539.560,-1061.298,2025-11-24T21:15:53Z
2,2024-01-01,SIN,SISTEMA INTERLIGADO NACIONAL,51266.264000,8982.283000,3205.270000,11.213,63465.030,0.000,2025-11-24T21:15:53Z
3,2024-01-01,SE,SUDESTE/CENTRO-OESTE,31710.361328,5046.854980,8.500000,8.663,34981.614,1792.766,2025-11-24T21:15:53Z
4,2024-01-01,S,SUL,13045.899414,979.270020,703.013977,1.000,9966.972,4762.211,2025-11-24T21:15:53Z


### 5. Carga para a Tabela de Staging
Limpa a tabela de staging e insere os novos dados lidos do Parquet. A tabela de staging serve como uma área intermediária antes da carga final.

In [6]:
# # Limpa staging antes de carregar novos dados
conn.execute("DELETE FROM balanco_subsistema_stg;")
conn.commit()

# Insere novos dados
df.to_sql("balanco_subsistema_stg", conn, if_exists="append", index=False)
conn.commit()

print(f"{len(df)} linhas carregadas na STAGING.")

43920 linhas carregadas na STAGING.


### 6. Verificação da Carga em Staging
Confere se os dados foram inseridos corretamente na tabela de staging, exibindo as primeiras linhas.

In [7]:
# Checar se os dados foram inseridos. Mostrar 5 primeiras linhas.
select_query = "SELECT * FROM balanco_subsistema_stg LIMIT 5;"
rows = conn.execute(select_query).fetchall()
for row in rows:
    print(row)

('2024-01-01 00:00:00', 'NE', 'NORDESTE', 3503.61791992, 486.47399902, 2491.56298828, 1.54999995, 11976.884, -5493.679, '2025-11-24T21:15:53Z')
('2024-01-01 00:00:00', 'N', 'NORTE', 3006.38500976, 2469.68408203, 2.19300007, 0.0, 6539.55999999, -1061.29799999, '2025-11-24T21:15:53Z')
('2024-01-01 00:00:00', 'SIN', 'SISTEMA INTERLIGADO NACIONAL', 51266.264, 8982.283, 3205.27, 11.213, 63465.03, 0.0, '2025-11-24T21:15:53Z')
('2024-01-01 00:00:00', 'SE', 'SUDESTE/CENTRO-OESTE', 31710.36132812, 5046.85498046, 8.5, 8.6630001, 34981.614, 1792.76599999, '2025-11-24T21:15:53Z')
('2024-01-01 00:00:00', 'S', 'SUL', 13045.89941406, 979.27001953, 703.01397705, 1.0, 9966.972, 4762.211, '2025-11-24T21:15:53Z')


### 7. Execução do UPSERT para a Camada Gold
Executa o script SQL que contém a lógica de `UPSERT` (INSERT ou UPDATE) para mover os dados da tabela de staging para a tabela `gold` final.

In [8]:
with open("sql/local/02_upsert_fact_energia.sql") as f:
    upsert_sql = f.read()

conn.executescript(upsert_sql)
conn.commit()

print("UPSERT finalizado na tabela GOLD!")

UPSERT finalizado na tabela GOLD!


### 8. Verificação da Carga na Tabela Gold
Valida se os dados foram persistidos corretamente na tabela `fact_balanco_subsistema`, consultando as primeiras linhas.

In [9]:
# Checar se os dados foram inseridos na tabela fact_balanco_subsistema. Mostrar 5 primeiras linhas.
select_gold_query = "SELECT * FROM fact_balanco_subsistema LIMIT 5;"
gold_rows = conn.execute(select_gold_query).fetchall()
for gold_row in gold_rows:
    print(gold_row)

('2024-01-01 00:00:00', 'NE', 'NORDESTE', 3503.61791992, 486.47399902, 2491.56298828, 1.54999995, 11976.884, -5493.679, '2025-11-24T21:15:53Z', '2025-11-24 21:16:12')
('2024-01-01 00:00:00', 'N', 'NORTE', 3006.38500976, 2469.68408203, 2.19300007, 0.0, 6539.55999999, -1061.29799999, '2025-11-24T21:15:53Z', '2025-11-24 21:16:12')
('2024-01-01 00:00:00', 'SIN', 'SISTEMA INTERLIGADO NACIONAL', 51266.264, 8982.283, 3205.27, 11.213, 63465.03, 0.0, '2025-11-24T21:15:53Z', '2025-11-24 21:16:12')
('2024-01-01 00:00:00', 'SE', 'SUDESTE/CENTRO-OESTE', 31710.36132812, 5046.85498046, 8.5, 8.6630001, 34981.614, 1792.76599999, '2025-11-24T21:15:53Z', '2025-11-24 21:16:12')
('2024-01-01 00:00:00', 'S', 'SUL', 13045.89941406, 979.27001953, 703.01397705, 1.0, 9966.972, 4762.211, '2025-11-24T21:15:53Z', '2025-11-24 21:16:12')
